In [59]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec as gs
from scipy.stats import ttest_ind

#### Load data

In [6]:
data = pd.read_excel('2022-08-24-msm-resR2-ref.xlsx',sheet_name='Results')

#### Calculate delta Ct

In [8]:
delta_ct=np.zeros(len(data))
for s in data['Sample_id'].unique():
    ctrl = data[(data['Target']=='MSMEG_2758')&(data['Sample_id']==s)]['CT'].mean()
    delta_ct[data['Sample_id']==s] = data[data['Sample_id']==s]['CT'].values-ctrl
data['d_Ct'] = delta_ct

#### Calculate delta-delta Ct and log2FC for selected bioreps

In [50]:
selected_samples = [17,19,21,22,27,28,29,30]
subset=[]
for s in data['Sample_id'].unique():
    if s in selected_samples:
        sample_d = data[data['Sample_id']==s].copy()
        for target in sample_d['Target'].unique():
            target_data = sample_d[sample_d['Target']==target].copy().reset_index(drop=True)
            median = np.median(target_data['d_Ct'])
            dCt2median = np.abs(target_data['d_Ct'].values-median)
            retained = np.where(dCt2median<0.25)[0]
            outlier_removed = target_data.iloc[retained].copy()
            subset.append(outlier_removed)        
subset = pd.concat(subset)

dd_Ct = []
for target in subset['Target'].unique():
    target_data = subset[subset['Target']==target].copy().reset_index(drop=True)
    ctrl = target_data[target_data['Sample_id']==17]['d_Ct'].median()
    target_data['dd_Ct'] = target_data['d_Ct']-ctrl
    target_data['Normalized expression'] = 2**(-target_data['dd_Ct'].values)
    target_data['Log2(fold change)'] = -target_data['dd_Ct'].values
    dd_Ct.append(target_data)
final = pd.concat(dd_Ct)

#### Export data (QL to plot)

In [87]:
final.to_excel('2022-08-24-msm-resR2-filtered.xlsx')